# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [29]:
# import numpy and pandas
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [30]:
# Your code here:
employee = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [31]:
employee.shape

(33183, 8)

In [32]:
# Your code here:
employee.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [33]:
# Your code here:
employee.isna().value_counts()



Name   Job Titles  Department  Full or Part-Time  Salary or Hourly  Typical Hours  Annual Salary  Hourly Rate
False  False       False       False              False             True           False          True           25161
                                                                    False          True           False           8022
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [34]:
# Your code here:
employee['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [35]:
# Your code here:
employee['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [36]:
employee.head(2)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN


In [37]:
# Your code here:
import scipy.stats as st

In [38]:
# 1. Set the hypothesis

# H0: mu != 30
# H1: mu == 30
n = 30

In [39]:
# 2. Choose significance / confidence level
a = 0.05

In [40]:
# 3. Sample
employee_hour = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate']
employee_hour

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [41]:
# 4. Compute statistic
# 5. Get p-value

statistic, p_value =st.ttest_1samp(employee_hour, n)

print(statistic)
print(p_value)

p_value < a

20.6198057854942
4.3230240486229894e-92


True

In [42]:
employee_hour.mean()

32.78855771628128

In [43]:
# print the results
print("t-statistic:", statistic)
print("p-value:", p_value)

t-statistic: 20.6198057854942
p-value: 4.3230240486229894e-92


In [44]:
# check if the p-value is less than the significance level (alpha)
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

Reject the null hypothesis


In [45]:
# 6. Decide
# if p value is lower that the significance level we reject the null 
# if p value is equal or bigger that the signiface level we do not reject 

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [46]:
t1 = employee[employee['Salary or Hourly'] == 'Salary']['Annual Salary']
t1.mean()

86786.99979015153

In [47]:
# set the null hypothesis value
n = 86000
a = 0.05

In [48]:
statistic, p_value =st.ttest_1samp(t1, n)

print(statistic)
print(p_value)

p_value < a

5.932870515690814
3.015758519916386e-09


True

Using the `crosstab` function, find the department that has the most hourly workers. 

In [49]:
employee.head(2)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN


In [50]:
# Your code here:
dept_hourly = pd.crosstab(employee['Department'], employee['Salary or Hourly'] == 'Hourly')-
# subset to only hourly workers
#dept_hourly = dept_hourly[dept_hourly[True] > 0]

# find the department with the most hourly workers
dept_most_hourly = dept_hourly[True]#.idxmax()

In [51]:
dept_most_hourly.sort_values(ascending=False)

Department
STREETS & SAN            1862
WATER MGMNT              1513
OEMC                     1273
AVIATION                 1082
GENERAL SERVICES          765
TRANSPORTN                725
PUBLIC LIBRARY            299
FAMILY & SUPPORT          287
CITY COUNCIL               64
FINANCE                    44
LAW                        40
ANIMAL CONTRL              19
POLICE                     10
MAYOR'S OFFICE              8
BUSINESS AFFAIRS            7
CULTURAL AFFAIRS            7
COMMUNITY DEVELOPMENT       4
HUMAN RESOURCES             4
HEALTH                      3
PROCUREMENT                 2
FIRE                        2
BUDGET & MGMT               2
HUMAN RELATIONS             0
INSPECTOR GEN               0
DoIT                        0
LICENSE APPL COMM           0
DISABILITIES                0
COPA                        0
CITY CLERK                  0
POLICE BOARD                0
BUILDINGS                   0
BOARD OF ETHICS             0
BOARD OF ELECTION           0

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [52]:
# Your code here:
n = 35
a = 0.05
streets = employee[employee['Department'] == 'STREETS & SAN']['Hourly Rate']
streets.dropna()

7        35.60
21       21.43
24       35.60
36       36.21
39       35.60
         ...  
33106    36.13
33107    35.60
33147    35.60
33149    36.21
33156    22.12
Name: Hourly Rate, Length: 1862, dtype: float64

In [53]:
statistic, p_value =st.ttest_1samp(streets, n)

print(statistic)
print(p_value)

p_value < a

nan
nan


False

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [54]:
employee.columns

Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')

In [70]:
a = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate']
mean = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate'].mean()
std = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate'].std()
n = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate'].count()
a

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [64]:
st.norm.interval(0.95,loc=mean,scale=std/np.sqrt(n))

(32.52349834813896, 33.05361708442361)

In [66]:
# Your code here:
st.sem(a, axis=0, ddof=1)

0.1352368565101596

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [72]:
employee.head(2)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN


In [75]:
# Your code here:
a = employee[employee['Department'] == 'POLICE']['Annual Salary']
mean = employee[employee['Department'] == 'POLICE']['Annual Salary'].mean()
std = employee[employee['Department'] == 'POLICE']['Annual Salary'].std()
n = employee[employee['Department'] == 'POLICE']['Annual Salary'].count()

display(a)

0        101442.0
1         94122.0
9         93354.0
11        84054.0
12        87006.0
           ...   
33177     72510.0
33178     72510.0
33179     48078.0
33180     90024.0
33181     93354.0
Name: Annual Salary, Length: 13414, dtype: float64

In [76]:
st.norm.interval(0.95,loc=mean,scale=std/np.sqrt(n))

(86177.08425202279, 86795.74475424399)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [28]:
# Your code here:

